In [29]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [30]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [31]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document

In [32]:
loader1 = TextLoader("./data/nlp-keywords.txt", encoding='utf-8')
loader2 = TextLoader("./data/book_document.txt", encoding='utf-8')

In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

In [34]:
# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

In [35]:
# 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [36]:
# DB 생성
db = FAISS.from_documents(
    documents=split_doc1+split_doc2, 
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

### **문서(Document)로부터 추가 (add_documents)**

`add_documents()` 메서드는 벡터 저장소에 문서를 추가하거나 업데이트하는 기능을 제공합니다.
<br>

**매개변수**

- `documents` (List[Document]): 벡터 저장소에 추가할 문서 리스트
- **kwargs: 추가 키워드 인자

In [37]:
# page_content, metadata 지정
db.add_documents(
    [
        Document(
            page_content="안녕하세요! 도큐먼트를 새로 추가해 보겠습니다.",
            metadata={"source": "mydata.txt"},
        )
    ],
    ids=["new_doc1"],
)

['new_doc1']

In [38]:
# 추가된 데이터를 확인
db.similarity_search("안녕하세요", k=1)

[Document(metadata={'source': 'mydata.txt'}, page_content='안녕하세요! 도큐먼트를 새로 추가해 보겠습니다.')]

In [39]:
# 추가된 데이터를 확인
db.index_to_docstore_id

{0: 'f976feb7-a57d-4d11-bb1c-565e9534c69d',
 1: 'bc7f1667-7e32-4a4f-8873-70241b87415c',
 2: '6ce3e6fb-7b7b-44f1-ad80-5604a6eadee8',
 3: '9f849899-2b4a-4174-8945-c997b0131e87',
 4: '3ecc373a-110d-47a0-bcd7-7cb4bc160656',
 5: '34a34471-a926-4413-a013-42e3429c73f9',
 6: '85223029-4483-4b65-82ba-86db7c3ccbf3',
 7: '4e7b04cd-4d5c-4d32-add4-674f58aa8e36',
 8: '74aab1d2-cd35-424c-a7cf-c8cb2ea048b2',
 9: '8d19783d-9518-4747-848b-3bfd28dcceb4',
 10: 'b661227e-68b2-4af0-b2bf-5db8bb2f320b',
 11: 'fdc7eb7a-3f99-4466-9bd6-b3f634931b38',
 12: 'b56e27df-de37-426b-b469-6dc82a0a3edb',
 13: '2df43a16-2f15-436e-8d87-6a8f601f0ca2',
 14: 'f6582ad2-0491-4953-ad13-2dd4920ba020',
 15: 'new_doc1'}

In [40]:
# id 로 삭제
db.delete(['new_doc1'])

True

In [41]:
# 삭제된 결과를 출력
db.index_to_docstore_id

{0: 'f976feb7-a57d-4d11-bb1c-565e9534c69d',
 1: 'bc7f1667-7e32-4a4f-8873-70241b87415c',
 2: '6ce3e6fb-7b7b-44f1-ad80-5604a6eadee8',
 3: '9f849899-2b4a-4174-8945-c997b0131e87',
 4: '3ecc373a-110d-47a0-bcd7-7cb4bc160656',
 5: '34a34471-a926-4413-a013-42e3429c73f9',
 6: '85223029-4483-4b65-82ba-86db7c3ccbf3',
 7: '4e7b04cd-4d5c-4d32-add4-674f58aa8e36',
 8: '74aab1d2-cd35-424c-a7cf-c8cb2ea048b2',
 9: '8d19783d-9518-4747-848b-3bfd28dcceb4',
 10: 'b661227e-68b2-4af0-b2bf-5db8bb2f320b',
 11: 'fdc7eb7a-3f99-4466-9bd6-b3f634931b38',
 12: 'b56e27df-de37-426b-b469-6dc82a0a3edb',
 13: '2df43a16-2f15-436e-8d87-6a8f601f0ca2',
 14: 'f6582ad2-0491-4953-ad13-2dd4920ba020'}

In [42]:
# 저장
db.save_local(folder_path="./faiss_db2", index_name="faiss_index2")

In [43]:
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="./faiss_db2",
    index_name="faiss_index2",
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),
    allow_dangerous_deserialization=True,
)

In [44]:
# 로드된 데이터를 확인
loaded_db.index_to_docstore_id

{0: 'f976feb7-a57d-4d11-bb1c-565e9534c69d',
 1: 'bc7f1667-7e32-4a4f-8873-70241b87415c',
 2: '6ce3e6fb-7b7b-44f1-ad80-5604a6eadee8',
 3: '9f849899-2b4a-4174-8945-c997b0131e87',
 4: '3ecc373a-110d-47a0-bcd7-7cb4bc160656',
 5: '34a34471-a926-4413-a013-42e3429c73f9',
 6: '85223029-4483-4b65-82ba-86db7c3ccbf3',
 7: '4e7b04cd-4d5c-4d32-add4-674f58aa8e36',
 8: '74aab1d2-cd35-424c-a7cf-c8cb2ea048b2',
 9: '8d19783d-9518-4747-848b-3bfd28dcceb4',
 10: 'b661227e-68b2-4af0-b2bf-5db8bb2f320b',
 11: 'fdc7eb7a-3f99-4466-9bd6-b3f634931b38',
 12: 'b56e27df-de37-426b-b469-6dc82a0a3edb',
 13: '2df43a16-2f15-436e-8d87-6a8f601f0ca2',
 14: 'f6582ad2-0491-4953-ad13-2dd4920ba020'}

In [45]:
# 검색기로 변환
retriever = loaded_db.as_retriever()

In [46]:
retriever.invoke('파이썬 책이 있나요?')

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열'),
 Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 손에잡히는 파이썬\n- 저자 : 문용준\n- 발행처 : BJpublic(비제이퍼블릭) 2018\n- isbn : 9791186697726\n- 청구기호 : 일 005.133-문66ㅅ\n- 등록번호 : CLP000178588\n- 소장위치 : 세 번째 책장 두 번 째 열\n\n\n### 제목 : 작심 3일 파이썬 Python\n- 저자 : 황덕창\n- 발행처 : 스포트라잇북

In [53]:
# MMR 검색 수행
retriever = db.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
)

In [48]:
retriever.invoke("파이썬 책이 있나요?")

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍

In [ ]:
# MMR 검색 수행. 상위 2개만 반환
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10})

In [50]:
retriever.invoke("파이썬 책이 있나요?")

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.\n연관키워드: 자연어 처리, 텍

In [51]:
# 가장 유사한 단일 문서만 검색
# k=1 로 설정하여 가장 유사한 문서만 검색
retriever = db.as_retriever(search_kwargs={"k": 1})

In [52]:
retriever.invoke("파이썬 책이 있나요?")

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열')]